In [1]:
# import important libraries
# Data analysis
import pandas as pd
import numpy as np
# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
# Machine Learning
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
# natural language toolkit
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
stop_words = stopwords.words('english')
import os
import gc
import re
import warnings
warnings.filterwarnings("ignore")
#pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_colwidth', -1)
from string import punctuation



/home/manoj/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package punkt to /home/manoj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/manoj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data
df = pd.read_csv('/home/manoj/Desktop/assignment/Consumer_Complaints.csv')

In [3]:
# Dataset top five rows
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/12/2014,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,M&T BANK CORPORATION,MI,48382,NaN,NaN,Referral,03/17/2014,Closed with explanation,Yes,No,759217
1,01/19/2017,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,When my loan was switched over to Navient i was never told that i had a deliquint balance because with XXXX i did not. When going to purchase a vehicle i discovered my credit score had been dropped from the XXXX into the XXXX. I have been faithful at paying my student loan. I was told that Navient was the company i had delinquency with. I contacted Navient to resolve this issue you and kept being told to just contact the credit bureaus and expalin the situation and maybe they could help me. I was so angry that i just hurried and paid the balance off and then after tried to dispute the delinquency with the credit bureaus. I have had so much trouble bringing my credit score back up.,NaN,"Navient Solutions, LLC.",LA,NaN,NaN,Consent provided,Web,01/19/2017,Closed with explanation,Yes,No,2296496
2,04/06/2018,Credit card or prepaid card,General-purpose credit card or charge card,"Other features, terms, or problems",Other problem,I tried to sign up for a spending monitoring program and Capital One will not let me access my account through them,NaN,CAPITAL ONE FINANCIAL CORPORATION,VA,NaN,Older American,Consent provided,Web,04/06/2018,Closed with explanation,Yes,NaN,2866101
3,06/08/2014,Credit card,NaN,Bankruptcy,NaN,NaN,NaN,AMERICAN EXPRESS COMPANY,ID,83854,Older American,NaN,Web,06/10/2014,Closed with explanation,Yes,Yes,885638
4,09/13/2014,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,NaN,NaN,"CITIBANK, N.A.",VA,23233,NaN,NaN,Web,09/13/2014,Closed with explanation,Yes,Yes,1027760


In [4]:
#  check information about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131011 entries, 0 to 1131010
Data columns (total 18 columns):
Date received                   1131011 non-null object
Product                         1131011 non-null object
Sub-product                     895843 non-null object
Issue                           1131011 non-null object
Sub-issue                       619937 non-null object
Consumer complaint narrative    326489 non-null object
Company public response         372501 non-null object
Company                         1131011 non-null object
State                           1116449 non-null object
ZIP code                        1044367 non-null object
Tags                            155170 non-null object
Consumer consent provided?      578882 non-null object
Submitted via                   1131011 non-null object
Date sent to company            1131011 non-null object
Company response to consumer    1131005 non-null object
Timely response?                1131011 non-null obje

In [8]:
df.describe(include ='all')

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
count,1131011,1131011,895843,1131011,619937,326489,372501,1131011,1116449,1044367,155170,578882,1131011,1131011,1131005,1131011,768536,1.131011e+06
unique,2496,18,76,166,218,314043,10,5017,63,25635,3,4,6,2445,8,2,2,NaN
top,09/08/2017,Mortgage,Credit reporting,"Loan modification,collection,foreclosure",Account status,There are many mistakes appear in my report without my understanding.,Company has responded to the consumer and the CFPB and chooses not to provide a public response,"EQUIFAX, INC.",CA,300XX,Older American,Consent provided,Web,09/08/2017,Closed with explanation,Yes,No,NaN
freq,3553,264785,152649,112313,37057,530,245591,95728,157087,5595,71597,326797,819493,3387,873051,1101244,620158,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.769509e+06
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.146791e+05
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.910445e+05
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.905926e+06
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.649432e+06


There are some null values in the dataset. we require Consumer complaint narrative as the features . I have choosen 'Product' as label for my convenience because the number of labels in 'Product' is less compared to 'Issues'

In [9]:
data = df[['Product','Consumer complaint narrative']]

In [10]:
# choose only some customer complaints as i have limited computation power.
data = data[:200000]

In [11]:
# Drop all those rows which have null values
data = data.dropna()

In [12]:
# group data by product
data.groupby('Product').groups

{'Bank account or service': Int64Index([   428,    462,    581,    673,    824,   1436,   1589,   2187,
               2402,   3436,
             ...
             198465, 198626, 198948, 199008, 199264, 199332, 199351, 199549,
             199700, 199935],
            dtype='int64', length=970),
 'Checking or savings account': Int64Index([   120,    193,    272,    361,    457,    467,    519,    662,
               1000,   1001,
             ...
             197851, 197871, 198406, 198559, 198678, 199204, 199434, 199444,
             199511, 199994],
            dtype='int64', length=1084),
 'Consumer Loan': Int64Index([   321,    622,    633,    671,   1033,   1351,   1456,   1585,
               2295,   2433,
             ...
             195472, 196811, 197071, 197894, 198635, 199200, 199735, 199801,
             199889, 199934],
            dtype='int64', length=658),
 'Credit card': Int64Index([   440,    472,    637,    893,    914,   1043,   1066,   1095,
               1318,  

In [13]:
# distribution of labels 
data['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    8183
Debt collection                                                                 7092
Mortgage                                                                        3935
Credit reporting                                                                2188
Credit card or prepaid card                                                     1791
Student loan                                                                    1703
Credit card                                                                     1292
Checking or savings account                                                     1084
Bank account or service                                                         970 
Consumer Loan                                                                   658 
Vehicle loan or lease                                                           509 
Money transfer, virtual currency, or money service               

In [14]:
# Count unique count in the labels
count_unique =data['Product'].value_counts()

In [15]:
# saving all labels name 
class_names = count_unique.index.values

In [16]:
# generating dictionary with labels name
dic = {index:i for i,index in enumerate(class_names)}
dic

{'Credit reporting, credit repair services, or other personal consumer reports': 0,
 'Debt collection': 1,
 'Mortgage': 2,
 'Credit reporting': 3,
 'Credit card or prepaid card': 4,
 'Student loan': 5,
 'Credit card': 6,
 'Checking or savings account': 7,
 'Bank account or service': 8,
 'Consumer Loan': 9,
 'Vehicle loan or lease': 10,
 'Money transfer, virtual currency, or money service': 11,
 'Payday loan, title loan, or personal loan': 12,
 'Payday loan': 13,
 'Prepaid card': 14,
 'Money transfers': 15,
 'Other financial service': 16,
 'Virtual currency': 17}

In [17]:
# map string labels into numeric using dictionary
data['labels'] = data['Product'].map(dic)

In [18]:
# Drop string label and now we have two columns in dataset.
# first is feature vector and other is numeric labels.
data = data.drop(['Product'],axis =1)

In [19]:
data.head()

,Consumer complaint narrative,labels
1,When my loan was switched over to Navient i was never told that i had a deliquint balance because with XXXX i did not. When going to purchase a vehicle i discovered my credit score had been dropped from the XXXX into the XXXX. I have been faithful at paying my student loan. I was told that Navient was the company i had delinquency with. I contacted Navient to resolve this issue you and kept being told to just contact the credit bureaus and expalin the situation and maybe they could help me. I was so angry that i just hurried and paid the balance off and then after tried to dispute the delinquency with the credit bureaus. I have had so much trouble bringing my credit score back up.,5
2,I tried to sign up for a spending monitoring program and Capital One will not let me access my account through them,4
7,"My mortgage is with BB & T Bank, recently I have been investigating ways to pay down my mortgage faster and I came across Biweekly Mortgage Calculator on BB & T 's website. It's a nice, easy to use calculator that you plug in your interest rate, mortgage amount, mortgage term, and payment type and it calculates your accelerated bi-weekly payment for you and shows you how much quicker you can pay down your loan. Ours figured out to pay off a 30 year mortgage in 26.4 years ... quite a savings! \nI called BB & T 's customer service number to inquire how I get set up on this payment plan. I was told they do not offer that type of payment plan, but I could send in my payments bi-weekly but it would not be applied until the full amount was received. ( the money would sit in a "" holding account '' until the full payment amount was collected ). I ended up calling back a few days later thinking the rep I was talking to didn't understand what I wanted to do or was not knowledgeable of this program. I got the SAME ANSWER! \nI then asked for the corporate BB & T office number where I could speak to someone that was knowledgeable of this product. After 3 days I received a phone call back from a corporate manager stating they do not offer this product, and they were "" checking into why this is on their website ''. She stated they do have a few customers that make bi-weekly payments, but they no longer offer this service. \nI don't understand how they can have this active link on their website under their Financial Planning Center tab to mislead customers when all they say is "" I'm sorry, I know you're upset about this '' Sounds like false advertising to me! \nhttps : //www.bbt.com/XXXX",2
13,"The entire lending experience with Citizens Bank was terrible. The lender kept pushing a XXXX ARM, which we had considered initially but decided to go with a 30yr fixed. When we received the closing disclosure, it was a XXXX ARM, we immediately requested that it be corrected to the 30yr fixed we decided upon but they kept delaying and saying it was changed but would n't send it to us. They waited until the next day and then tried to convince us to waive our rights so it would n't push the closing date due to the 3 day clock reset. This seemed very intentional, like it was something they often do, because they sent us an email instructing us to draft a letter : -- -- From the loan officer : I have outlined the basis of the letter in this email. Please elaborate on why you wanted the product changed and make sure to include your contact information at the bottom as they will need this. \n"" To Whom it may concern, This letter is to waive my right to review the new Closing Disclosure 3 business days prior to closing. I must close on the property at ( property address ) or I face financial hardship in the form of losing my Earnest Money Deposit and property falling through. I must close on this property on XXXX. There is substantial expense associated with my drive and move to ( property address ) and require this property so I may begin ( new job ) on time. \n\nIn addition this letter is to address the change in ra

In [20]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [21]:
# clean data
data['consumer complain']=clean_review(data['Consumer complaint narrative'].values)

In [22]:
# obtain labels from dataset
target=data['labels'].values
y=to_categorical(target)

In [23]:
# divide the dataset into training and validation dataset.
X_train_text,X_val_text,y_train,y_val=train_test_split(data['Consumer complaint narrative'].values,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

(24463,) (24463, 18)
(6116,) (6116, 18)


In [24]:
# finding the unique words in data
all_words=' '.join(X_train_text)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

51417

In [25]:
# Finding max length of a complain
r_len=[]
for text in X_train_text:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

5277

In [26]:
# LSTM model
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 3
num_classes=18

In [27]:
# Tokenize text
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)



In [28]:
# Sequence padding
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
print(X_train.shape,X_val.shape)

(24463, 5277) (6116, 5277)


In [29]:
print(y_train.shape,y_val.shape)

(24463, 18) (6116, 18)


In [30]:
model1=Sequential()
model1.add(Embedding(max_features,10,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 10)          514170    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 64)          19200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 18)                594       
Total params: 546,380
Trainable params: 546,380
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit data in LSTM model.
%%time
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 13419 samples, validate on 3355 samples
Epoch 1/3
13419/13419 [==============================] - 1391s 104ms/step - loss: 2.3023 - acc: 0.2882 - val_loss: 2.1332 - val_acc: 0.2951
Epoch 2/3
11264/13419 [========================>.....] - ETA: 3:05 - loss: 2.1329 - acc: 0.2900

Could not run for more epoch due to limited computation power.